# Student Loan Risk with Deep Learning

In [18]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y.sample(5)

895     1
152     0
540     0
1570    1
331     1
Name: credit_ranking, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
X.sample(5)

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
1172,9.7,0.420,0.46,2.1,0.074,5.0,16.0,0.99649,3.27,0.74,12.30
1542,6.7,0.855,0.02,1.9,0.064,29.0,38.0,0.99472,3.30,0.56,10.75
478,9.6,0.680,0.24,2.2,0.087,5.0,28.0,0.99880,3.14,0.60,10.20
93,7.7,0.490,0.26,1.9,0.062,9.0,31.0,0.99660,3.39,0.64,9.60
836,6.7,0.280,0.28,2.4,0.012,36.0,100.0,0.99064,3.26,0.39,11.70


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Confirmation
print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

Training set size: (1279, 11)
Testing set size: (320, 11)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
# This seems wrong. This is a good way to introduce leakage. Forgive the additional code.
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Test the bad boy
print(f"Sample of the training data:\n {X_train_scaled[:5]}")


Sample of the training data:
 [[-0.73307913  0.6648928  -1.25704443 -0.3204585  -0.45362151 -0.74240736
  -0.6455073   0.24000129  0.98846046  0.0630946  -0.87223395]
 [ 1.06774091 -0.62346154  1.52314768  0.60886277 -0.36954631 -1.12518952
  -1.11200285  0.18789883 -1.7535127  -0.17390392 -0.77978452]
 [-1.74604041 -1.07158479 -1.35814232 -0.53491726 -0.78992229  1.07580793
   0.53628144 -2.67773653  2.32756363  0.77409018  3.28799021]
 [-0.62052788  0.49684658 -1.05484864 -0.0345135  -0.20139592  0.11885252
   1.18937522  0.37546769  1.24352773 -0.76640023 -0.6873351 ]
 [-0.50797663  0.60887739 -1.00429969 -0.53491726  0.26101766 -0.74240736
  -0.7077067  -0.33312578 -0.09557544 -0.47015208 -0.77978452]]


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
input_features = X_train_scaled.shape[1]

# Review the number of features
print(f"Number of input features: {input_features}")

Number of input features: 11


In [10]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = input_features * 2

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = input_features

# Define the number of neurons in the output layer
output_neurons = 1

# Confirmation statements
print(f"Input Features: {input_features}")
print(f"Hidden Layer 1 Nodes: {hidden_nodes_layer1}")
print(f"Hidden Layer 2 Nodes: {hidden_nodes_layer2}")
print(f"Output Layer Neurons: {output_neurons}")

Input Features: 11
Hidden Layer 1 Nodes: 22
Hidden Layer 2 Nodes: 11
Output Layer Neurons: 1


In [11]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Add the second hidden layer
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_neurons, activation='sigmoid'))

/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# Display the Sequential model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 22)             │           264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │           253 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 529 (2.07 KB)

 Trainable params: 529 (2.07 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

Epoch 1/50


40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5177 - loss: 0.7060 - val_accuracy: 0.6281 - val_loss: 0.6399
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6836 - loss: 0.6150 - val_accuracy: 0.7125 - val_loss: 0.5838
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7133 - loss: 0.5735 - val_accuracy: 0.7344 - val_loss: 0.5489
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7118 - loss: 0.5394 - val_accuracy: 0.7406 - val_loss: 0.5316
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7335 - loss: 0.5110 - val_accuracy: 0.7281 - val_loss: 0.5255
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7271 - loss: 0.5117 - val_accuracy: 0.7437 - val_loss: 0.5226
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7337 - loss: 0.5118 - val_accuracy: 0.7344 - val_loss: 0.5211
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7340 - loss: 0.5114 - val_accuracy: 0.7375 - val_loss: 0.5195
Epo

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7735 - loss: 0.4764 
Test Loss: 0.5187271237373352
Test Accuracy: 0.7437499761581421


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path
model_file_path = 'student_loans.keras'

# Export your model to a keras file
model.save(model_file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [20]:
# Set the model's file path
this_seems_silly = 'student_loans.keras'

# Load the model to a new object
loaded_model = load_model(this_seems_silly)

print(loaded_model)

<Sequential name=sequential, built=True>


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [22]:
# Make predictions with the test data
predictions = model.predict(X_test_scaled)

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Probability'])

# Display a sample of the predictions
print(predictions_df.head())


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
   Probability
0     0.632540
1     0.372665
2     0.735528
3     0.733726
4     0.985361


In [23]:
# Save the predictions to a DataFrame and round the predictions to binary results

# We are roundin'!
binary_predictions = (predictions > 0.5).astype(int)

# Now, we're savin'!
predictions_df = pd.DataFrame(binary_predictions, columns=['Predicted_class'])

# Now, we're printin'!
print(predictions_df)

     Predicted_class
0                  1
1                  0
2                  1
3                  1
4                  1
..               ...
315                1
316                0
317                0
318                1
319                1

[320 rows x 1 columns]


### Step 4: Display a classification report with the y test data and predictions

In [24]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, binary_predictions))

              precision    recall  f1-score   support

           0       0.72      0.77      0.74       154
           1       0.77      0.72      0.74       166

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.75      0.74      0.74       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

|1. **Data for Recommendation System**:
   To build a recommendation system for student loan options, you'd need:
   - **Student Financial Profiles**: Information on income, existing debts, and financial need.
   - **Academic Data**: GPA, major, and educational institution to assess potential future earnings and loan repayment capacity.
   - **Loan Details**: Interest rates, repayment terms, and conditions from various loan providers.
   - **Historical Repayment Data**: How previous students with similar profiles managed their loans.
   This data is relevant as it helps predict the suitability of different loan options based on financial capability and academic trajectory, aligning loan offers with the student’s ability to repay.

2. **Type of Filtering Method**:
   - The proposed system would likely benefit from **content-based filtering**. This approach can utilize the specific attributes of both the students (financial status, academic performance) and the loan products (terms, rates) to make personalized recommendations. 
   - Content-based filtering is suitable here because it allows for personalized recommendations based on individual user profiles and specific item features, which is crucial in matching students with the most fitting loan options based on their unique circumstances.

3. **Real-world Challenges**:
   - **Data Privacy and Security**: Handling sensitive financial and personal information requires robust security measures to protect against data breaches. Privacy is a major concern because the recommendation system will process highly sensitive personal and financial information.
   - **Bias and Fairness**: Ensuring the recommendation system does not inherit or amplify biases present in historical data, which could lead to unfair loan recommendations. This is crucial in avoiding discriminatory practices and ensuring equitable access to loan options for all students, regardless of background.
